In [15]:
pip install empath

Note: you may need to restart the kernel to use updated packages.


In [3]:
from empath import Empath
lexicon = Empath()

# score means number of words in the input text that match the words associated with that category

In [6]:
lexicon.analyze("he hit the other person", categories=["violence"])
# => {'violence': 1.0}

{'violence': 1.0}

# Normalize: divide by the total number of words in the text

In [7]:
lexicon.analyze("he hit the other person", categories=["violence"], normalize=True)
# => {'violence': 0.2}

{'violence': 0.2}

# Fail to connect with external server

In [ ]:
#lexicon.create_category("colors",["red","blue","green"])
# => ["blue", "green", "purple", "purple", "green", "yellow", "red", "grey", "violet", "gray", "blue", "orange", "white", "pink", "yellow", "black", "brown", "brown", "red", "aqua", "turquoise", "blue_color", "colored", "color", "same_shade", "violet", "gray", "grey", "teal", "nice_shade", "coloured", "forest_green", "colored", "different_shade", "colour", "sparkly", "reddish", "beautiful_shade", "greenish", "indigo", "darker_shade", "emerald", "lovely_shade", "tints", "crimson", "dark_purple", "pink", "emerald", "sapphire", "golden", "lighter_shade", "lime_green", "coloured", "bright", "same_color", "specks", "red", "golden_color", "different_shades", "chocolate_brown", "orange", "bluish", "green", "deep_purple", "magenta", "green_color", "dark_shade", "bright_orange", "milky", "lilac", "light_brown", "sparkling", "golden_brown", "silvery", "baby_blue", "blood_red", "pink", "teal", "blue", "yellowish", "turquoise", "same_colour", "sparkly", "aquamarine", "black_color", "white", "cerulean", "perfect_shade", "dark", "speckled", "charcoal", "greyish", "midnight_blue", "emerald_green", "deep_brown", "ocean_blue", "flecks", "amber", "pinkish", "jet_black"]

#lexicon.analyze("My favorite color is blue", categories=["colors"], normalize=True)
# => {'colors': 0.4}

# Trial 1: one paragraph

In [16]:
pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.6 MB 9.1 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/11.6 MB 8.7 MB/s eta 0:00:01
   ------------------- -------------------- 5.8/11.6 MB 9.3 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.6 MB 9.9 MB/s eta 0:00:01
   --------------------------------- ------ 9.7/11.6 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 9.1 MB/s eta 0:00:00
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd

# Perform sentiment analysis
result = lexicon.analyze(
    "Despite decades of scientific consensus on the importance of urgent climate action, a significant subset of global citizens continues to be doubtful, even dismissive of climate change science. Among these, there has lately been a growing contingent – individuals who have not necessarily declared allegiance to the contrarian side en masse, but instead display scattered, piecemeal scepticism on this increasingly pressing issue. Weary to unravel the complexities surrounding their doubt-some views, The NZ Herald initiated an in-depth inquiry; endeavouring to delve deeper and dispel the haze enveloping understanding of climate change amongst New Zealand communities.", 
    normalize=True
)

# Convert results to a pandas DataFrame
df = pd.DataFrame(list(result.items()), columns=['Category', 'Score'])

# Filter for scores greater than 0
filtered_df = df[df['Score'] > 0.0]

# Sort the DataFrame by the Score column in descending order
sorted_df = filtered_df.sort_values(by='Score', ascending=False)

# Display the sorted DataFrame
print(sorted_df)


             Category     Score
8                cold  0.031579
103           weather  0.031579
136            warmth  0.031579
3               money  0.021053
25         government  0.021053
58       irritability  0.010526
101          sympathy  0.010526
105             trust  0.010526
114        politeness  0.010526
35             school  0.010526
127              gain  0.010526
132           science  0.010526
151          politics  0.010526
156        technology  0.010526
158    shape_and_size  0.010526
163         terrorism  0.010526
175         messaging  0.010526
192  positive_emotion  0.010526


# Trial 2.1: dataset of twitter posts on climate change from Kaggle -  tweet by tweet

In [24]:
import pandas as pd

# Load the dataset of Twitter posts
df_tweets = pd.read_csv('twitter_sentiment_data.csv')

# Sample 100 tweets from the dataset
df_sample = df_tweets.sample(100)

# Create an empty list to hold the raw analysis results
analysis_results = []

# Loop through each tweet and analyze it using Empath
for tweet in df_sample['message']:  
    result = lexicon.analyze(tweet, normalize=True)
    
    # Append the result (raw analysis) to the results list
    analysis_results.append(result)

# Now convert the raw results to a DataFrame and filter out categories with score 0
all_data = []  # List to hold the final data for DataFrame

for tweet, result in zip(df_sample['message'], analysis_results):
    # Convert the result into a DataFrame
    temp_df = pd.DataFrame(list(result.items()), columns=['Category', 'Score'])
    
    # Filter out categories with score 0
    filtered_df = temp_df[temp_df['Score'] > 0.0]
    
    # Sort the filtered DataFrame by Score in descending order
    sorted_df = filtered_df.sort_values(by='Score', ascending=False)
    
    # Add a new column 'Tweet' to the sorted DataFrame for the corresponding tweet text
    sorted_df['Tweet'] = tweet
    
    # Append the sorted DataFrame to the all_data list
    all_data.append(sorted_df)

# Concatenate all individual DataFrames into one
final_df = pd.concat(all_data, ignore_index=True)

# Optionally, save the results to a CSV file
#final_df.to_csv('climate_change_analysis_results.csv', index=False)

# Display the final result
print(final_df.head(20))



      Category     Score                                              Tweet
0   government  0.090909  RT @csmonitor: Could mutant plants save us fro...
1    superhero  0.090909  RT @csmonitor: Could mutant plants save us fro...
2      healing  0.090909  RT @csmonitor: Could mutant plants save us fro...
3       heroic  0.090909  RT @csmonitor: Could mutant plants save us fro...
4        trust  0.090909  RT @csmonitor: Could mutant plants save us fro...
5    terrorism  0.090909  RT @csmonitor: Could mutant plants save us fro...
6      monster  0.090909  RT @csmonitor: Could mutant plants save us fro...
7       giving  0.090909  RT @csmonitor: Could mutant plants save us fro...
8        money  0.041667  RT @SethMacFarlane: HRC proposes installing ha...
9        sleep  0.041667  RT @SethMacFarlane: HRC proposes installing ha...
10        cold  0.041667  RT @SethMacFarlane: HRC proposes installing ha...
11     weather  0.041667  RT @SethMacFarlane: HRC proposes installing ha...
12      warm

# Trial 2.2: dataset of twitter posts on climate change from Kaggle - aggregated scores of all the post text

In [25]:
import pandas as pd

# Load the dataset of Twitter posts
df_tweets = pd.read_csv('twitter_sentiment_data.csv')

df_sample = df_tweets.sample(100)  # Sampling 100 tweets for the analysis

# Create an empty dictionary to hold the total scores for each category
total_scores = {}

# Loop through each tweet and analyze it using Empath
for tweet in df_sample['message']:  
    result = lexicon.analyze(tweet, normalize=False)
    
    # Convert the result into a DataFrame and filter out categories with score 0
    temp_df = pd.DataFrame(list(result.items()), columns=['Category', 'Score'])
    filtered_df = temp_df[temp_df['Score'] > 0.0]
    
    # Add the scores to the total_scores dictionary
    for category, score in filtered_df.values:
        if category in total_scores:
            total_scores[category] += score  # Accumulate the score for each category
        else:
            total_scores[category] = score  # Initialize the score for new category

# Convert the total_scores dictionary into a DataFrame
aggregated_df = pd.DataFrame(list(total_scores.items()), columns=['Category', 'Total_Score'])

# Sort the DataFrame by Total_Score in descending order and get the top 20 categories
top_20_df = aggregated_df.sort_values(by='Total_Score', ascending=False).head(20)

# Display the top 20 topics
print(top_20_df)


            Category  Total_Score
7             warmth         93.0
1               cold         78.0
5            weather         76.0
0              money         59.0
2         government         36.0
24         terrorism         29.0
9   negative_emotion         15.0
60     communication         13.0
62          speaking         11.0
19               war         11.0
39          business         10.0
18             fight          9.0
81           meeting          8.0
41            school          8.0
63            listen          8.0
83         messaging          8.0
35            injury          8.0
73           science          7.0
40              work          7.0
59           college          7.0
